In [1]:
# coding=utf-8
import sys

sys.path.append('../')

from util import load_json
from data_loader.song_info_loader import SongMetaInfoLoader

from tqdm.auto import tqdm
import numpy as np
import sys
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
      # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [2]:
TRAIN_DATA_PATH = '../res/orig/train.json'
META_DATA_PATH = '../res/song_meta.json'

In [3]:
train_data = load_json(TRAIN_DATA_PATH)
meta_data = SongMetaInfoLoader()
meta_data.make_info_map(playlist_data=train_data, meta_data=load_json(META_DATA_PATH))

make meta info of songs!



done!


In [4]:
def playlist_to_aids(playlists):
    aid_list = []
    for plays in tqdm(playlists):
        aids = list(map(lambda s: str(meta_data.song_album_id_map[s]), plays['songs']))
        aid_list.append(aids)
        
    return aid_list

In [5]:
aid_seqs = playlist_to_aids(playlists=train_data)

In [6]:
train_tags = [tag for tags in map(lambda e: filter(lambda t: t, e['tags']), train_data) for tag in tags]

In [7]:
tag_set = set(train_tags)

In [8]:
len(tag_set)

25479

In [9]:
from collections import Counter

In [10]:
def freq_in_seq(counts, ratio_thres=0.9, print_freq=1000):
    total = sum(map(lambda kc:kc[1], counts))
    acc = 0
    for cnt_idx in range(0, len(counts)):
        val, cnt = counts[cnt_idx]
        acc += cnt
        ratio = acc / total
        if ratio < ratio_thres:
            continue
        if cnt_idx % print_freq == 0:
            print(f"{cnt_idx}, {val}, {cnt}, {ratio * 100:.3f}")
    #     break

In [11]:
aid_counter = Counter([aid for sub_seq in aid_seqs for aid in sub_seq]).most_common()

In [12]:
freq_in_seq(counts=aid_counter, print_freq=5000)

60000, 2328588, 8, 90.124
65000, 10077224, 7, 91.015
70000, 4805, 6, 91.806
75000, 2296096, 6, 92.514
80000, 2657488, 5, 93.124
85000, 589457, 5, 93.714
90000, 2647078, 4, 94.195
95000, 2304888, 4, 94.666
100000, 10362390, 3, 95.105
105000, 10332514, 3, 95.459
110000, 10413570, 3, 95.812
115000, 192854, 3, 96.166
120000, 10211449, 2, 96.479
125000, 2320238, 2, 96.715
130000, 10320115, 2, 96.951
135000, 352962, 2, 97.187
140000, 2278278, 2, 97.422
145000, 2291333, 2, 97.658
150000, 2175387, 2, 97.894
155000, 2058349, 1, 98.071
160000, 1179551, 1, 98.189
165000, 4148327, 1, 98.307
170000, 10296046, 1, 98.425
175000, 2647114, 1, 98.543
180000, 10340819, 1, 98.661
185000, 1180461, 1, 98.779
190000, 10209324, 1, 98.897
195000, 1087970, 1, 99.014
200000, 10224706, 1, 99.132
205000, 10214779, 1, 99.250
210000, 1307780, 1, 99.368
215000, 10268675, 1, 99.486
220000, 116133, 1, 99.604
225000, 101746, 1, 99.722
230000, 2297489, 1, 99.840
235000, 741858, 1, 99.958


In [13]:
tag_counter = Counter(train_tags).most_common()

In [14]:
freq_in_seq(counts=tag_counter, print_freq=100)

2900, 퇴근_후, 7, 90.060
3000, 기분좋아, 7, 90.244
3100, 신나는팝, 7, 90.428
3200, 옛날생각, 6, 90.598
3300, 연간차트, 6, 90.756
3400, 오늘은_뭐듣지, 6, 90.913
3500, 정신수양요가음악, 6, 91.071
3600, 해질무렵, 5, 91.216
3700, 멋짐주의, 5, 91.348
3800, 한강에서, 5, 91.479
3900, 봄산책, 5, 91.610
4000, 남부힙합, 5, 91.742
4100, 토이, 5, 91.873
4200, 그때, 4, 92.001
4300, 들어본적있는노래, 4, 92.106
4400, 침대위에서듣는음악, 4, 92.211
4500, 첸백시, 4, 92.316
4600, 추억에, 4, 92.421
4700, 걸그룹모음, 4, 92.526
4800, 생각_사색_명상, 4, 92.631
4900, 데뷔앨범, 4, 92.736
5000, 세계, 4, 92.841
5100, 가을겨울, 4, 92.946
5200, 타임슬립, 3, 93.025
5300, 몸이들썩들썩, 3, 93.104
5400, GROOVY한, 3, 93.183
5500, 산책하면서, 3, 93.261
5600, 생각했던대로, 3, 93.340
5700, 마블유니버스, 3, 93.419
5800, 돋보이는, 3, 93.498
5900, 월드디제이페스티벌, 3, 93.577
6000, 객원보컬, 3, 93.655
6100, electro, 3, 93.734
6200, 훈스디제이, 3, 93.813
6300, 라운지_바, 3, 93.892
6400, 비오는밤, 3, 93.970
6500, 슬픔감성, 3, 94.049
6600, 푸디토리움, 3, 94.128
6700, 앰비션뮤직, 2, 94.185
6800, 건반, 2, 94.237
6900, 이것이다, 2, 94.290
7000, 흥나는EDM, 2, 94.342
7100, 케빈오, 2, 94.395
7200, 커버영상, 2, 94.447

In [15]:
# tf.keras.preprocessing.text.Tokenizer?

In [16]:
aid_cnt_thres = 60000
aid_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=aid_cnt_thres)

In [17]:
aid_tokenizer.fit_on_texts(aid_seqs)

In [18]:
aid_tokenizer.texts_to_sequences(['2302415 2326467 714453 862952 860568 4234823'])

[[255, 363, 6269, 2536, 1332]]

In [19]:
aid_tokenizer.num_words

60000

In [20]:
for e in tag_counter:
    if e[0] == '':
        print(e)

In [21]:
tag_cnt_thres = 3000
candidate_tags = set(map(lambda tc:tc[0], tag_counter[:tag_cnt_thres]))
filtered_train = []
for train in train_data:
    valid_tags = filter(lambda e: e in candidate_tags, train['tags'])
    train['tags'] = list(valid_tags)
    if len(train['tags']) > 0:
        filtered_train.append(train)
# filtered_train = list(filter(lambda t: len(list(filter(lambda e: e in candidate_tags, t['tags']))) > 0, train_data))

In [22]:
filtered_train[0]

{'tags': ['힐링', '휴식', '밤', '새벽'],
 'id': 147668,
 'plylst_title': 'To. 힘들고 지친 분들에게',
 'songs': [663185,
  649626,
  6855,
  188486,
  348451,
  169945,
  512599,
  532114,
  454528,
  418935,
  124485,
  517372,
  549950,
  540588,
  500931,
  233641,
  331055,
  490266,
  268515,
  531820,
  413762,
  422713,
  215080,
  413189,
  577903,
  352228,
  630395,
  539109,
  152475,
  111865,
  7460,
  72432,
  572480,
  348092,
  324208,
  186039,
  376140,
  270269,
  622615,
  35001,
  444706,
  491303,
  408698,
  325979,
  25538,
  549392,
  473514,
  666814,
  118223,
  697100,
  333034,
  359279,
  421124,
  403253,
  27784,
  118049,
  339124,
  175073,
  522895,
  6925,
  615815,
  672550,
  379112,
  80972,
  227036,
  112153],
 'like_cnt': 12,
 'updt_date': '2016-06-23 10:06:27.000'}

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer

In [24]:
tag_encoder = MultiLabelBinarizer()

In [25]:
tag_encoder.fit(map(lambda e: e['tags'], filtered_train))

MultiLabelBinarizer(classes=None, sparse_output=False)

In [26]:
len(tag_encoder.classes_)

3000

In [27]:
tag_encoder.classes_

array(['00년대', '0살', '1', ..., '힙합음악', '힙합클럽', '힙합트랙'], dtype=object)

In [28]:
def make_x_y(dict_list, max_seq):
    aids_list = playlist_to_aids(dict_list)
    tag_list = list(map(lambda e: e['tags'], dict_list))
    x_list = tf.keras.preprocessing.sequence.pad_sequences(sequences=aid_tokenizer.texts_to_sequences(aids_list), 
                                                           maxlen=max_seq, padding='pre')
    y_list = tag_encoder.transform(tag_list)
    
    return x_list, y_list

In [29]:
x, y = make_x_y(dict_list=filtered_train, max_seq=32)

In [30]:
print(x[0])
print(x.shape)

[  122   120  8601  1088  1758   266  2088  1259  1306  3921    89  1439
   249   280   109  1850   420  5552  5042  1292 10161   605   255   980
  2075 17076  3068   272   363  7469  3688   258]
(90143, 32)


In [31]:
print(np.sum(y))
print(np.sum(y[0]))
print(y.shape)

343636
4
(90143, 3000)


In [32]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
class AlbumEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)

    def call(self, x):
        x = self.token_emb(x)
        return x

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.1)

In [35]:
x_train.shape

(81128, 32)

In [36]:
max_seq = 32
embed_dim = 8  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 4  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_seq,))
embedding_layer = TokenAndPositionEmbedding(max_seq, aid_cnt_thres, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(len(tag_encoder.classes_), activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
token_and_position_embedding (None, 32, 8)             480256    
_________________________________________________________________
transformer_block (Transform (None, 32, 8)             396       
_________________________________________________________________
global_average_pooling1d (Gl (None, 8)                 0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                576       
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0     

In [38]:
optim = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
model.compile(metrics=["accuracy"], optimizer=optim, loss=keras.losses.CategoricalCrossentropy())

In [39]:
history = model.fit(
    x_train, y_train, batch_size=64, epochs=10, validation_data=(x_dev, y_dev)
)

Train on 81128 samples, validate on 9015 samples
Epoch 1/10
81128/81128 [==============================] - 19s 235us/sample - loss: 22.2827 - accuracy: 0.0899 - val_loss: 21.6100 - val_accuracy: 0.1100
Epoch 2/10
81128/81128 [==============================] - 16s 194us/sample - loss: 20.8959 - accuracy: 0.1177 - val_loss: 20.9696 - val_accuracy: 0.1380
Epoch 3/10
81128/81128 [==============================] - 16s 195us/sample - loss: 20.3441 - accuracy: 0.1439 - val_loss: 20.7425 - val_accuracy: 0.1449
Epoch 4/10
81128/81128 [==============================] - 16s 196us/sample - loss: 20.0307 - accuracy: 0.1549 - val_loss: 20.6393 - val_accuracy: 0.1530
Epoch 5/10
81128/81128 [==============================] - 16s 196us/sample - loss: 19.8003 - accuracy: 0.1677 - val_loss: 20.5669 - val_accuracy: 0.1598
Epoch 6/10
81128/81128 [==============================] - 16s 195us/sample - loss: 19.6400 - accuracy: 0.1780 - val_loss: 20.3787 - val_accuracy: 0.1752
Epoch 7/10
81128/81128 [=========

In [ ]:
from sacred import Experiment
from sacred.observers import MongoObserver

In [ ]:
ex = Experiment('kakao-arena', interactive=True)
mongo_url = '172.28.10.4:27017'  # Or <server-static-ip>:<port> if running on server
ex.observers.append(MongoObserver.create(url=mongo_url,
                                         db_name='omniboard'))

In [ ]:
@ex.config
def model_config():
    embed_dim = 32  # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer